In [74]:
import numpy as np
import pandas as pd
from google.cloud import bigquery
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import plotly.express as px
import db_dtypes
import bigframes.pandas as bpd
from IPython.display import display, HTML
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import math
import statsmodels.api as sm
from datetime import datetime
from typing import Union
import logging
import sys

ppu = pd.read_pickle("/Users/shazeb.asad/Downloads/results_all_ppu.pickle")
opu = pd.read_pickle("/Users/shazeb.asad/Downloads/results_all_opu.pickle")

filtered_df_ppu = ppu[ppu['is_sign_corrected'] == True]
#filtered_df_ppu = ppu.copy()


grouped_ppu = filtered_df_ppu.groupby('entity_id')['mean_diff'].mean()

filtered_df_opu = opu[opu['is_sign_corrected'] == True]
#filtered_df_opu = opu.copy()


grouped_opu = filtered_df_opu.groupby('entity_id')['mean_diff'].mean()

merged_df = pd.merge(
    grouped_ppu.rename('mean_diff_ppu'),
    grouped_opu.rename('mean_diff_opu'),
    left_index=True,
    right_index=True,
    how='outer' 
).reset_index()


tmp = """

with all_tests as (
select des.entity_id 
      ,des.country_code
      ,des.test_name
      ,des.test_start_date
      ,des.test_end_date
      ,des.objective
      ,des.hypothesis
from `fulfillment-dwh-production.cl.dps_experiment_setups` des
where des.test_start_date >= '2024-01-01'
and des.experiment_type = 'AB'
--and des.test_name = 'PA_20230108_R_B0_P_Vendor_Clustering_ene_2024'
group by all
), 
significance as (
SELECT st.country_code
      ,st.test_name
      ,st.group_a
      ,st.group_b
      ,kpi_label
      ,mean_a
      ,mean_b
      ,LEAST(p_value, corrected_p_value, cuped_p_value, corrected_cuped_p_value) AS min_p_value
      ,CASE 
      WHEN LEAST(p_value, corrected_p_value, cuped_p_value, corrected_cuped_p_value) <= 0.05 THEN 1 
      ELSE 0 
      END AS significance
      ,(mean_b - mean_a) AS incremental
      ,p_value
      ,corrected_p_value
      ,cuped_p_value
      ,corrected_cuped_p_value
FROM `fulfillment-dwh-production.rl.dps_ab_test_significance_dataset` st
where st.kpi_label IN ('orders_per_user', 'profit_local_per_user')
and st.group_a like "%Control%"
and st.group_b like "%Variation%"
AND label = 'All'
),
tmp as (
SELECT ate.country_code
      ,ate.entity_id
      ,ate.test_name
      ,ate.objective
      ,ate.hypothesis
      ,group_b variation
      ,max(case when kpi_label = 'orders_per_user' then mean_a else null end) AS orders_per_user_control
      ,max(case when kpi_label = 'orders_per_user' then incremental else null end) AS incremental_orders_per_user
      ,max(case when kpi_label = 'orders_per_user' then significance else null end) AS significance_orders_per_user
      ,max(case when kpi_label in ('profit_local_per_user') then mean_a else null end) AS profit_per_user_control
      ,max(case when kpi_label in ('profit_local_per_user') then incremental else null end) AS incremental_profit_per_user
      ,max(case when kpi_label in ('profit_local_per_user') then significance else null end) AS significance_profit_per_user
from all_tests ate
join significance s on s.country_code = ate.country_code and s.test_name = ate.test_name
group by 1,2,3,4,5,6
),
ranked_data AS (
SELECT *,
      ROW_NUMBER() OVER (
      PARTITION BY entity_id, test_name
      ORDER BY 
            CASE 
            WHEN significance_orders_per_user = 1 AND significance_profit_per_user = 1 THEN 1
            WHEN significance_profit_per_user = 1 THEN 2
            WHEN significance_orders_per_user = 1 THEN 3
            WHEN incremental_orders_per_user > 0 AND incremental_profit_per_user > 0 THEN 4
            ELSE 5
            END ASC,
            CASE 
            WHEN incremental_orders_per_user > 0 AND incremental_profit_per_user > 0 THEN NULL
            ELSE incremental_profit_per_user
            END DESC,
            significance_orders_per_user DESC
) AS rank
FROM tmp
)
SELECT  *
FROM ranked_data
where rank = 1
order by entity_id, test_name

"""

In [75]:
ppu

,entity_id,test_id_1,test_id_2,total_users_in_both,same_control,same_variation,variation_control,control_variation,same_variant_both_tests,overlap_ratio,...,is_sign,p_values_corrected,is_sign_corrected,mean_diff,p_value_welch,p_value_student,is_sign_welch,is_sign_student,is_sign_welch_corr,is_sign_student_corr
0,AP_PA,104,107,14436,1462,5680,5843,1451,7142,0.49,...,False,0.916379,False,0.495967,2.607247e-01,2.206525e-01,False,False,False,False
4,AP_PA,104,131,5087,509,1996,2073,509,2505,0.49,...,False,0.908600,False,0.413720,5.678392e-01,5.261211e-01,False,False,False,False
6,AP_PA,104,145,3658,384,1471,1434,369,1855,0.51,...,False,0.835544,False,0.209330,8.373828e-01,8.257001e-01,False,False,False,False
8,AP_PA,104,149,3548,378,1437,1382,351,1815,0.51,...,False,0.567537,False,1.297873,1.246024e-01,1.147003e-01,False,False,False,False
10,AP_PA,105,106,141773,10698,82711,34252,14112,93409,0.66,...,True,0.000000,True,0.292210,2.039038e-01,1.484399e-01,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25490,YS_TR,252,259,1137654,106188,546731,273831,210904,652919,0.57,...,False,0.640439,False,-0.094083,7.271601e-03,2.792988e-03,True,True,True,True
25492,YS_TR,252,260,758314,52768,410390,137321,157835,463158,0.61,...,False,0.992505,False,0.261815,1.420667e-08,2.548095e-11,True,True,True,True
25494,YS_TR,252,263,879074,75593,434090,217100,152291,509683,0.58,...,False,0.489018,False,-0.103580,1.030041e-02,3.447575e-03,True,True,True,True
25496,YS_TR,252,264,1204716,111411,580923,289921,222461,692334,0.57,...,False,0.819911,False,0.169402,3.935162e-09,1.257652e-10,True,True,True,True


In [76]:
tt = ppu[ppu['is_sign_corrected'] == True]

In [77]:
tt[tt['entity_id'] == "AP_PA"]

,entity_id,test_id_1,test_id_2,total_users_in_both,same_control,same_variation,variation_control,control_variation,same_variant_both_tests,overlap_ratio,...,is_sign,p_values_corrected,is_sign_corrected,mean_diff,p_value_welch,p_value_student,is_sign_welch,is_sign_student,is_sign_welch_corr,is_sign_student_corr
10,AP_PA,105,106,141773,10698,82711,34252,14112,93409,0.66,...,True,0.000000e+00,True,0.292210,2.039038e-01,1.484399e-01,False,False,False,False
14,AP_PA,105,108,9722,630,5724,2428,940,6354,0.65,...,True,4.597807e-15,True,0.429442,3.362726e-01,2.849849e-01,False,False,False,False
20,AP_PA,105,111,41186,3112,23671,10081,4322,26783,0.65,...,True,1.217179e-88,True,0.230100,2.120604e-01,1.450838e-01,False,False,False,False
22,AP_PA,105,112,29331,1757,17001,7541,3032,18758,0.64,...,True,3.304315e-15,True,0.372457,8.336731e-02,5.400949e-02,False,False,False,False
24,AP_PA,105,113,164046,25335,83995,52617,2099,109330,0.67,...,True,0.000000e+00,True,0.589723,6.562591e-05,8.781118e-06,True,True,True,True
27,AP_PA,105,117,99334,13939,68767,14812,1816,82706,0.83,...,True,0.000000e+00,True,0.384965,1.005070e-01,6.274657e-02,False,False,False,False
29,AP_PA,105,128,83993,7052,35500,34576,6865,42552,0.51,...,True,2.255969e-02,True,0.125955,4.695302e-01,4.520315e-01,False,False,False,False
31,AP_PA,105,129,19428,1703,8295,7873,1557,9998,0.51,...,True,1.312126e-03,True,0.832641,1.048204e-04,1.180293e-04,True,True,True,True
33,AP_PA,105,130,103766,14629,72373,14755,2009,87002,0.84,...,True,0.000000e+00,True,2.204961,0.000000e+00,0.000000e+00,True,True,True,True
37,AP_PA,105,132,81675,4572,47298,20730,9075,51870,0.64,...,True,1.737648e-11,True,0.514929,1.235097e-02,6.565808e-03,True,True,True,True


In [78]:
project_id = "logistics-customer-staging"
logging.info(f"Initializing BigQuery client for project: {project_id}")

try:
    # Initialize BigQuery client
    client = bigquery.Client(project=project_id)
except Exception as e:
    logging.error(f"Failed to initialize BigQuery client: {e}")
    sys.exit(1)


data_test = client.query(tmp).to_dataframe()

/Users/shazeb.asad/global_pricing/venv_bayesian/lib/python3.13/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [79]:
data_test

,country_code,entity_id,test_name,objective,hypothesis,variation,orders_per_user_control,incremental_orders_per_user,significance_orders_per_user,profit_per_user_control,incremental_profit_per_user,significance_profit_per_user,rank
0,pa,AP_PA,DH Prueba AB AA - David v3,Other,Test SB and AB Test at the same time,Variation1,0.861605,0.049459,1,1.289803,0.054743,0,1
1,pa,AP_PA,PA_20230108_R_B0_P_Vendor_Clustering_ene_2024,Orders,Test Vendor Clustering to get order incrementa...,Variation1,1.009419,-0.001168,0,2.196321,-0.109286,1,1
2,pa,AP_PA,PA_20240116_Z_Z0_O_BV_DMarts_Pa,Orders,Test BV elimination in DMarts,Variation1,0.155505,0.001454,0,0.321016,0.003085,0,1
3,pa,AP_PA,PA_20240131_R_H0_O_Sevice_Fee_Growth_PA,Orders,Test Service Fee Elasticity,Variation2,1.373508,-0.011190,0,1.553298,0.100032,1,1
4,pa,AP_PA,PA_20240221_D_B0_P_DMarts_Chorrera_DF_Increase,Profit,Test DF increase to understand user elasticity,Variation1,0.147425,-0.002640,0,0.202263,0.031107,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1592,tr,YS_TR,TR_20241209_D_A0_O_Market_Test_V2_,Orders,to see the effect & changes of FDF- DF in orders,Variation2,0.141040,0.003301,1,13.547729,-0.711015,1,1
1593,tr,YS_TR,TR_20241210_D_C0_R_MOV_150-120-100_V2_,Orders,to see the effect & changes MOV in orders and ...,Variation1,0.140720,0.002049,0,13.429885,-0.124594,0,1
1594,tr,YS_TR,TR_20250221_R_B0_O_Istanbul_SBX,Orders,actions gain MS,Variation1,0.941125,0.000758,0,55.917749,0.160843,0,1
1595,tr,YS_TR,TR_20250221_R_B0_O_ROC_SBX,Orders,gain MS,Variation1,0.765333,-0.001065,0,31.655723,0.480579,1,1


In [80]:
merged_final = data_test.merge(
    merged_df,
    left_on='entity_id',
    right_on = 'entity_id',
    how='left'
).reset_index()

In [81]:
# Step 1: Calculate the result of the subtraction
merged_final['diff'] = merged_final['incremental_orders_per_user'] - merged_final['mean_diff_opu']

# Step 2: Check if the sign changed
merged_final['sign_change'] = (
    np.sign(merged_final['incremental_orders_per_user']) != np.sign(merged_final['diff'])
)



In [82]:
# Group by entity_id and calculate counts
sign_change_stats = merged_final.groupby('entity_id').agg(
    total_rows=('sign_change', 'count'),
    sign_change_count=('sign_change', 'sum')
)

# Optional: Calculate percentage of sign changes
sign_change_stats['sign_change_ratio'] = (
    sign_change_stats['sign_change_count'] / sign_change_stats['total_rows']
)


In [83]:
tt = sign_change_stats.reset_index()

In [84]:
tt.to_csv("tt_output.csv", index=False)

In [86]:
tt['sign_change_ratio'].mean()

np.float64(0.5573371375800898)